# DDPG PROJECT

Import gym and define pendulum environment

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gym
import numpy as np
from matplotlib import pyplot as plt

#env = gym.make('Pendulum-v1', g=9.81, render_mode = 'human')

Check API-conformity

In [ ]:
from gym.utils.env_checker import check_env

Import helpers

In [ ]:
from helpers import *

## 3) Heuristic policy

##### Wrap environment in a NormalizedEnv class

In [ ]:
env = gym.make('Pendulum-v1', g=9.81)
#env = gym.make('Pendulum-v1', g=9.81, render_mode = 'human')
env

In [ ]:
env = NormalizedEnv(env)
env

##### RandomAgent

In [ ]:
random_agent = RandomAgent(env)

In [ ]:
rewards = np.zeros((10,200))

for i in range(10):
    state = env.reset()
    state = state[0]
    
    trunc = False
    cur_reward = []
    
    while not trunc:   
        action = random_agent.compute_action(state) 
        (next_state, reward, term, trunc, info) = env.step(action)
        state = next_state
        cur_reward.append(reward)
        
        if term or trunc:
            observation, info = env.reset()
    
    rewards[i] = cur_reward

rand_rewards = rewards

##### Heuristic pendulum agent

In [ ]:
heur_agent = HeuristicPendulumAgent(env, 0.8)
rewards = np.zeros((10,200))

for i in range(10):
    state = env.reset()
    state = state[0]

    trunc = False
    cur_reward = []

    while not trunc:
        action = heur_agent.compute_action(state) 
        (next_state, reward, term, trunc, info) = env.step(action)
        state = next_state
        cur_reward.append(reward)

        if term or trunc:
            observation, info = env.reset()
            
    rewards[i] = cur_reward

heur_rewards = rewards

plt.figure()
plt.plot(np.arange(200), np.mean(rand_rewards, axis = 0))
plt.plot(np.arange(200), np.mean(heur_rewards, axis = 0))
plt.legend(['Random','Heuristic'])

Effect of fixed torque

In [ ]:
torques = np.linspace(-1,1,30)
avg_rewards = []

for t in torques:
    heur_agent = HeuristicPendulumAgent(env,t)
    rewards = []

    for _ in range(10):
        state = env.reset()
        state = state[0]

        trunc = False
        cur_reward = 0

        while not trunc:
            action = heur_agent.compute_action(state) 
            (next_state, reward, term, trunc, info) = env.step(action)
            state = next_state
            cur_reward += reward

            if term or trunc:
                observation, info = env.reset()

        rewards.append(cur_reward)
    avg_rewards.append(np.mean(rewards))

plt.figure()
plt.plot(torques, avg_rewards)
plt.xlabel('Fixed torque')
plt.ylabel('Avg. total reward (over 10 runs)')

In [ ]:
# env.close()

## 4) Q function of the heuristic policy

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def train_epoch(model, agent, optimizer, criterion, gamma, Buffer, epoch, device):
    model.eval()
    state = env.reset()
    state = state[0]
    Trunc = False
    loss_total = []
    while not Trunc:
        with torch.no_grad():
            action = agent.compute_action(state) 
        (next_state, reward, term, Trunc, info) = env.step(action)
        Buffer.store_transition((state, action, reward, next_state, Trunc))
        state = next_state
        #print(Buffer.buffer[-1])
        if (len(Buffer.buffer) >= 128):
            model.train()
            b = Buffer.batch_buffer(128)
            data = np.zeros((128,4))
            reward = np.zeros(128)
            next_data = np.zeros((128,4))
            trunc = np.zeros(128)
            for i,transition in enumerate(b):  
                data[i] = np.concatenate((transition[0], transition[1]))
                reward[i] = transition[2]
                trunc[i] = transition[4]
                with torch.no_grad():           
                    next_data[i] = np.concatenate((transition[3], agent.compute_action(transition[3])))

            data = torch.tensor(data, dtype=torch.float32).to(device)   
            reward = torch.tensor(reward, dtype=torch.float32).to(device)
            next_data = torch.tensor(next_data, dtype=torch.float32).to(device)
            trunc = torch.tensor(trunc, dtype = torch.bool)

            optimizer.zero_grad()
            output = model(data)  # (state, action) -> Q(s, a)
            #print(output)
            with torch.no_grad():
                target = (reward + gamma*model(next_data).flatten()).reshape(128,1)
            #print(target)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            loss_total.append(loss.item())
            
    
    return loss_total

In [ ]:
heur_agent = HeuristicPendulumAgent(env,0.8)
max_size = 1e4

device = "cpu"
model = QNetwork().to(device)
lr = 1e-4
gamma = 0.99
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
criterion = torch.nn.MSELoss()
num_epochs = 2
history = []
Buffer = ReplayBuffer(max_size = max_size)

for epoch in range(num_epochs):
    loss = train_epoch(model, heur_agent, optimizer, criterion, gamma, Buffer, epoch, device)
    if epoch%10 == 0:
        print(loss[-1])
    history.extend(loss)
    

In [ ]:
len(history)

In [ ]:
plt.plot(np.arange(len(history)), history)
plt.xlabel('Step')
plt.ylabel('MSE Loss')

In [ ]:
def compute_reward(alpha, alpha_speed, torque):
    return -(alpha**2 + 0.1*alpha_speed**2 + 0.001*torque**2)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

logs = [(0,0),(0,1),(2,1),(-2,1),(-2,-1)]

for l in logs:
    speed = l[0]
    torque = l[1]
    
    fig = plt.figure(figsize=(20, 5))
    
    rad = np.linspace(0, 5, 100)
    azm = np.linspace(-np.pi, np.pi, 100)
    r, th = np.meshgrid(rad, azm)
    
    # Q-Network
    speed_array = speed*np.ones(r.shape)
    torque_array = torque*np.ones(r.shape)
    data = np.stack([np.cos(th), np.sin(th), speed_array, torque_array], axis = 2)
    data = torch.tensor(data, dtype = torch.float32)
    z = model(data)
    z = z.detach().numpy().squeeze()

    ax1 = fig.add_subplot(121, projection="polar")
    ax1.grid(False)
    pc = ax1.pcolormesh(th, r, z)
    ax1.plot(azm, r, color='k', ls='none') 
    ax1.set_theta_zero_location('N')
    fig.colorbar(pc)

    # Original heuristic
    rad = np.linspace(0, 5, 100)
    azm = np.linspace(-np.pi, np.pi, 100)
    r, th = np.meshgrid(rad, azm)
    z_heur = compute_reward(th, speed, torque*np.sign(np.cos(th))*np.sign(speed))
    
    ax2= fig.add_subplot(122,projection="polar")
    ax2.grid(False)
    pc2 = ax2.pcolormesh(th, r, z_heur)
    ax2.plot(azm, r, color='k', ls='none') 
    ax2.set_theta_zero_location('N')
    fig.colorbar(pc2)

## 5) Minimal Implementation of DDPG

In [ ]:
def compute_loss_policy(state, action, model_Q):
    #with torch.no_grad():
    #model_Q.eval()
    #action.detach().numpy()  
    data = torch.cat((state, action), axis = 1)
    #data = torch.tensor(data, dtype=torch.float32).to(device)   
    loss_policy = torch.mean(-model_Q(data))
    #loss_policy.requires_grad = True
    return loss_policy


def train_epoch_policy(model_policy, model_Q, agent, optimizer, criterion, gamma, Buffer, epoch, device):
    state = env.reset()
    state = state[0]

    Trunc = False
    loss_total = []
    while not Trunc:
        model_policy.eval()
        with torch.no_grad():
            action = agent.compute_action(state)
        (next_state, reward, term, Trunc, info) = env.step(action)
        Buffer.store_transition((state, action, reward, next_state, Trunc))
        state = next_state
    
        if len(Buffer.buffer) >= 128:
            model_Q.eval()
            model_policy.train()
            b = Buffer.batch_buffer(128)
            data = np.zeros((128,3))
            reward = np.zeros(128)
            next_data = np.zeros((128,3))
            trunc = np.zeros(128)
            for i,transition in enumerate(b):        
                data[i] = transition[0]
                reward[i] = transition[2]
                trunc[i] = transition[4]
                next_data[i] = transition[3]

            data = torch.tensor(data, dtype=torch.float32).to(device)   
            reward = torch.tensor(reward, dtype=torch.float32).to(device)
            next_data = torch.tensor(next_data, dtype=torch.float32).to(device)
            trunc = torch.tensor(trunc, dtype = torch.bool)
            optimizer.zero_grad()
            output = model_policy(data)  # state -> action
            
            loss = compute_loss_policy(data, output, model_Q)
            # loss = criterion(loss_p, torch.zeros([128,1], requires_grad = True))
            loss.backward()
            optimizer.step()
            loss_total.append(loss.item())

    return loss_total

In [ ]:
device = "cpu"

# Define DDPG agent
noise_std = 0.3
ddpg_agent = DDPGAgent(noise_std, device)

 
# model_Q = QNetwork().to(device)
# model_policy = PolicyNetwork().to(device)

max_size = 1e5
Buffer = ReplayBuffer(max_size = max_size)


# Training parameters, ecc...
lr = 1e-4
gamma = 0.99
optimizer_Q = torch.optim.Adam(ddpg_agent.Q_network.parameters(), lr = lr)
optimizer_policy = torch.optim.Adam(ddpg_agent.policy_network.parameters(), lr = lr)
criterion_Q = torch.nn.MSELoss()
criterion_policy = torch.nn.L1Loss()
num_epochs = 5 # metti 1000

history_Q = []
history_policy = []

state = env.reset()
state = state[0]

for epoch in range(num_epochs):
    
    action = ddpg_agent.compute_action(state, deterministic = False) 
    (next_state, reward, term, trunc, info) = env.step(action)
    Buffer.store_transition((state, action, reward, next_state, trunc))
    state = next_state

    if term or trunc:
        observation, info = env.reset()
        # Save reward of episode
    
    loss_Q = train_epoch(ddpg_agent.Q_network, ddpg_agent, optimizer_Q, criterion_Q, gamma, Buffer, epoch, device)
    loss_policy = train_epoch_policy(ddpg_agent.policy_network, ddpg_agent.Q_network, ddpg_agent, optimizer_policy, criterion_policy, gamma, Buffer, epoch, device)
    
    if epoch%10 == 0:
        print('lq = ', loss_Q[-1])
        print('lp = ', loss_policy[-1])
    history_Q.extend(loss_Q)
    history_policy.extend(loss_policy)
   

In [ ]:
plt.figure()
plt.plot(history_Q)
plt.plot(history_policy)
plt.show()

Simulation

In [ ]:
# env = gym.make('Pendulum-v1', g=9.81, render_mode = 'human')

# rewards = np.zeros((5,200))

# for i in range(5):
#     state = env.reset()
#     state = state[0]
    
#     trunc = False
#     cur_reward = []
    
#     while not trunc:   
#         action = ddpg_agent.compute_action(model_policy, state, deterministic = True) 
#         (next_state, reward, term, trunc, info) = env.step(action)
#         state = next_state
#         cur_reward.append(reward)
        
#         if term or trunc:
#             observation, info = env.reset()
    
#     rewards[i] = cur_reward

# rand_rewards = rewards

In [ ]:
# env = gym.make('Pendulum-v1', g=9.81)
rewards = np.zeros((100,200))

#Buffer = ReplayBuffer(max_size = 400)

for i in range(100):
    state = env.reset()
    state = state[0]

    trunc = False
    cur_reward = []

    while not trunc:
        action = ddpg_agent.compute_action(state, deterministic = True) 
        (next_state, reward, term, trunc, info) = env.step(action)
        state = next_state
        cur_reward.append(reward)

        if term or trunc:
            observation, info = env.reset()
            
    rewards[i] = cur_reward

ddpg_rewards = rewards

plt.figure()
plt.plot(np.arange(200), np.mean(rand_rewards,axis = 0))
plt.plot(np.arange(200), np.mean(heur_rewards, axis = 0))
plt.plot(np.arange(200), np.mean(ddpg_rewards, axis = 0))

plt.legend(['Random','Heuristic','DDPG'])

In [ ]:
logs = [(0,0),(0,1),(2,1),(-2,1),(-2,-1)]

for l in logs:
    speed = l[0]
    torque = l[1]
    
    fig = plt.figure(figsize=(20, 5))
    
    rad = np.linspace(0, 5, 100)
    azm = np.linspace(-np.pi, np.pi, 100)
    r, th = np.meshgrid(rad, azm)
    
    # Q-Network
    speed_array = speed*np.ones(r.shape)
    torque_array = torque*np.ones(r.shape)
    data = np.stack([np.cos(th), np.sin(th), speed_array, torque_array], axis = 2)
    data = torch.tensor(data, dtype = torch.float32)
    z = model_Q(data)
    z = z.detach().numpy().squeeze()
    
    ax1 = fig.add_subplot(121, projection="polar")
    ax1.grid(False)
    pc = ax1.pcolormesh(th, r, z)
    ax1.plot(azm, r, color='k', ls='none') 
    ax1.set_theta_zero_location('N')
    fig.colorbar(pc)

# 6) Target Networks

In [ ]:
def train_epoch_target(model, target_model, agent, optimizer, criterion, gamma, Buffer, epoch, device):
    model.train()
    b = Buffer.batch_buffer(128)
    data = np.zeros((128,4))
    reward = np.zeros(128)
    next_data = np.zeros((128,4))
    trunc = np.zeros(128)
    for i,transition in enumerate(b):        
        data[i] = np.concatenate([transition[0], transition[1]])
        reward[i] = transition[2]
        trunc[i] = transition[4]
        next_data[i] = np.concatenate([transition[3], agent.compute_action(transition[3])])
    
    data = torch.tensor(data, dtype=torch.float32).to(device)   
    reward = torch.tensor(reward, dtype=torch.float32).to(device)
    next_data = torch.tensor(next_data, dtype=torch.float32).to(device)
    trunc = torch.tensor(trunc, dtype = torch.bool)
    
    optimizer.zero_grad()
    output = model(data)  # (state, action) -> Q
    with torch.no_grad():
        target = (reward + trunc*target_model(next_data).flatten()).reshape(128,1)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    
    return loss


def train_epoch_policy_target(model_policy, model_Q, target_model_policy, target_model_Q, optimizer, criterion, gamma, Buffer, epoch, device):
    
    model_policy.train()
    b = Buffer.batch_buffer(128)
    data = np.zeros((128,3))
    reward = np.zeros(128)
    next_data = np.zeros((128,3))
    trunc = np.zeros(128)
    for i,transition in enumerate(b):        
        data[i] = transition[0]
        reward[i] = transition[2]
        trunc[i] = transition[4]
        next_data[i] = transition[3]
    
    data = torch.tensor(data, dtype=torch.float32).to(device)   
    reward = torch.tensor(reward, dtype=torch.float32).to(device)
    next_data = torch.tensor(next_data, dtype=torch.float32).to(device)
    trunc = torch.tensor(trunc, dtype = torch.bool)
    optimizer.zero_grad()
    output = model_policy(data)  # state -> action
    loss = compute_loss_policy(data, output, target_model_Q)
    # loss = criterion(loss_p, torch.zeros([128,1], requires_grad = True))
    loss.backward()
    optimizer.step()
    
    return loss

In [ ]:
device = "cpu"

# Define DDPG agent
noise_std = 0.3
TAU = [0.1]

for tau in TAU:
    ddpg_agent = DDPGAgent(noise_std, tau)

    # 
    # model_Q = QNetwork().to(device)
    # model_policy = PolicyNetwork().to(device)

    # Training parameters, ecc...
    lr = 1e-4
    gamma = 0.99
    optimizer_Q = torch.optim.Adam(ddpg_agent.Q_network.parameters(), lr = lr)
    optimizer_policy = torch.optim.Adam(ddpg_agent.policy_network.parameters(), lr = lr)
    criterion_Q = torch.nn.MSELoss()
    criterion_policy = torch.nn.L1Loss()
    num_epochs = 2000 # metti 1000

    history_Q = []
    history_policy = []

    state = env.reset()
    state = state[0]

    for epoch in range(num_epochs):

        action = ddpg_agent.compute_action(state, deterministic = False) 
        (next_state, reward, term, trunc, info) = env.step(action)
        Buffer.store_transition((state, action, reward, next_state, trunc))
        state = next_state

        if term or trunc:
            observation, info = env.reset()
            # Save reward of episode

        loss_Q = train_epoch_target(ddpg_agent.Q_network, ddpg_agent.target_Q_network, ddpg_agent, optimizer_Q, criterion_Q, gamma, Buffer, epoch, device)
        loss_policy = train_epoch_policy_target(ddpg_agent.policy_network, ddpg_agent.Q_network, ddpg_agent.target_policy_network, ddpg_agent.target_Q_network, optimizer_policy, criterion_policy, gamma, Buffer, epoch, device)

        ddpg_agent.update_target_params

        if epoch%100 == 0:
            print(loss_Q)
            print(loss_policy)
        history_Q.append(loss_Q.detach().numpy())
        history_policy.append(loss_policy.detach().numpy())


In [ ]:
plt.figure()
plt.plot(np.arange(num_epochs), history_Q)
plt.plot(np.arange(num_epochs), history_policy)

In [ ]:
# env = gym.make('Pendulum-v1', g=9.81)
rewards = np.zeros((100,200))

#Buffer = ReplayBuffer(max_size = 400)

for i in range(100):
    state = env.reset()
    state = state[0]

    trunc = False
    cur_reward = []

    while not trunc:
        action = ddpg_agent.compute_action(state, deterministic = True) 
        (next_state, reward, term, trunc, info) = env.step(action)
        state = next_state
        cur_reward.append(reward)

        if term or trunc:
            observation, info = env.reset()
            
    rewards[i] = cur_reward

ddpg_target_rewards = rewards

plt.figure()
plt.plot(np.arange(200), np.mean(rand_rewards,axis = 0))
plt.plot(np.arange(200), np.mean(heur_rewards, axis = 0))
plt.plot(np.arange(200), np.mean(ddpg_rewards, axis = 0))
plt.plot(np.arange(200), np.mean(ddpg_target_rewards, axis = 0))


plt.legend(['Random','Heuristic','DDPG','DDPG with targets'])

# 7) Ornstein-Uhlenbeck Noise